In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
import json

In [2]:
load_dotenv(override=True)
key = os.getenv("OPEN_API_KEY")
openai = OpenAI()
model = "gpt-4o-mini"

In [14]:
system_message = "You are a helpful assistant specializing in selling mobile phones with your expert salesman skills who highlights the key features of a mobile phone. If a user asks for mobile price, provide that according the model. If you don't know the price or don't have any information about what is asked, just say that."

In [4]:
price_function = {
    "name": "get_mobile_phone_price",
    "description": "Get the price of a mobile phone as per model.",
    "parameters": {
        "type": "object",
        "properties": {
            "mobile_model": {
                "type": "string",
                "description": "The mobile phone that the customer wants to buy",
            },
        },
        "required": ["mobile_model"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]

In [15]:
def chat(message, history):
    formatted_history = []
    for user_msg, bot_msg in history:
        formatted_history.append({"role": "user", "content": user_msg})
        if bot_msg:  
            formatted_history.append({"role": "assistant", "content": bot_msg})
    
    messages = [{"role": "system", "content": system_message}] + formatted_history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model = model, messages = messages, tools = tools)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=model, messages=messages)
    
    return response.choices[0].message.content
    

In [16]:
import sqlite3
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (mobile TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [17]:
def set_mobile_price(mobile, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (mobile, price) VALUES (?, ?) ON CONFLICT(mobile) DO UPDATE SET price = ?', (mobile.lower(), price, price))
        conn.commit()

In [18]:
mobile_prices = {"iphone 10":799, "Galaxy s1": 899, "iphone 14 xs": 1420, "Google Pixel One": 2999}
for mobile_model, price in mobile_prices.items():
    set_mobile_price(mobile_model, price)

In [19]:
def get_mobile_phone_price(mobile_model):
    print(f"DATABASE TOOL CALLED: Getting price for {mobile_model}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE mobile = ?', (mobile_model.lower(),))
        result = cursor.fetchone()
        return f"Mobile price to {mobile_model} is ${result[0]}" if result else "No price data available for this phone"

In [20]:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_mobile_phone_price":
        arguments = json.loads(tool_call.function.arguments)
        mobile_model = arguments.get('mobile_model')
        price_details = get_mobile_phone_price(mobile_model)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [ ]:
gr.ChatInterface(fn = chat).launch()

/Users/simranjaura/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting price for iphone 10
DATABASE TOOL CALLED: Getting price for Galaxy S10
